In [102]:
import requests
import string
import pickle
from html.parser import HTMLParser

from bs4 import BeautifulSoup
import pandas as pd

In [83]:
pd.options.display.max_colwidth = 1000

In [110]:
# scrapping artists

url_dict = {}
count = 0
for letter in string.ascii_lowercase:
    letter_list = []
    
    r = requests.get("https://ukutabs.com/{}/".format(letter))
    if r.status_code != 200:
        raise Exception("Wrong Response.")
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    for li in soup.find_all('li', class_='cat-item'):
        for a in li.find_all('a'):
            count += 1
            letter_list.append(a['href'])
    url_dict[letter] = letter_list
print('Found {} artists'.format(count))

Found 1883 artists


In [114]:
# utility functions
import json
def save_json(data, name):
    with open('obj/'+ name + '.json', 'w') as fp:
        json.dump(data, fp)
        
def load_json(name):
    with open('obj/'+ name + '.json', 'r') as fp:
        data = json.load(fp)
    return data

In [113]:
save_json(url_dict, 'artists')

In [121]:
# scraping songs
artists = load_json('artists')
songs = {}

for letter, l in artists.items():
    for url in l:
        name = url.split('/')[-2]
        

In [81]:
# parser object to parse one song
class MyHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.recording = False
        self.current_tag = []
        
        self.chords = []
        self.chords_line = []
        self.chords_part = []
        self.lyrics = []
        self.lyrics_part = []
        self.song = None
        self.artist = None
        self.genre = []
        self.parts = []
        
    def handle_starttag(self, tag, attrs):
        self.current_tag.append(tag)
        if tag == 'pre':
            self.recording = True

    def handle_endtag(self, tag):
        if tag == 'pre':
            self.recording = False
            self.handle_last_batch()
        self.current_tag.pop()
        

    def handle_data(self, data):
        if self.recording:
            if self.current_tag[-1] == 'strong':
                # a new part of song
                if self.current_tag[-2] == 'strong':
                    # chords may be nested in <strong>
                    whole_name = ''.join([self.parts.pop(), data])
                    self.parts.append(whole_name)
                else:
                    self.parts.append(data)
                    
                if len(self.lyrics_part) != 0 or len(self.chords_part) != 0:
                    # append if not the first part
                    self.lyrics.append(self.lyrics_part[:])
                    self.chords.append(self.chords_part[:])
                    self.lyrics_part.clear()
                    self.chords_part.clear()
            if self.current_tag[-1] == 'a' and data.strip() != '':
                # a new chord
                self.chords_line.append(data)
            if self.current_tag[-1] == 'pre' and data.strip() != '':
                # one line of music finished
                # clean data
                words = data.split()
                lyrics = ' '.join(words)
                if lyrics != '' and lyrics != ':':
                    self.lyrics_part.append(lyrics)
                if len(self.chords_line) != 0:
                    self.chords_part.append(self.chords_line[:])
                    self.chords_line.clear()
        
    def handle_last_batch(self):
        self.lyrics.append(self.lyrics_part)
        self.chords.append(self.chords_part)
        
    def dump(self):
        #print(len(self.chords), len(self.parts), len(self.lyrics))
        assert(len(self.chords) == len(self.parts))
        assert(len(self.lyrics) == len(self.parts))
        return {'song': self.song, 'artist': self.artist, 'genre': self.genre, 'chords': self.chords, 'lyrics': self.lyrics, 'parts': self.parts}

In [82]:
df = pd.DataFrame(columns=['song', 'artist', 'genre', 'parts', 'chords', 'lyrics'])

parser = MyHTMLParser()
parser.feed(r.text)
df = df.append(parser.dump(), ignore_index=True)

In [78]:
df

,song,artist,genre,parts,chords,lyrics
0,None,None,[],"[Verse, Chorus, Post-Chorus, Verse, Chorus, Bridge, Chorus, Outro]","[[[E, B, Dbm, A], [E, B, Dbm, A], [E, B, Dbm, A, B], [E, B, Dbm, A], [E], [B, Dbm, A, B]], [[A, B, E, B, A], [B, E, B], [A, B, E, B, A, B]], [[E, B, Dbm, A]], [[E, B, Dbm, A], [E, B, Dbm, A, B], [E, B, Dbm, A], [E, B, Dbm], [A, B]], [[A, B, E, B, A], [B, E, B], [A, B, E, B, A, B]], [[E], [B, Dbm], [A, E], [B], [Dbm], [A, B]], [[A, B, E, B, A], [B, E, B], [A, B, E, B, A, B, E, B], [A, B, E, B, A, B]], [[E, B, Dbm, A, B], [E, B, Dbm, A, B], [E]]]","[[Hey-ay, Hey-ay-AY-ay-ay, Hey-ay-AY-ay-ay, Your lipstick stains on the front lobe of my left side brains, I knew I wouldn't for-get you and so I went and let you blow my mind, Your sweet moonbeam the smell of you in every single dream I dream, I knew when we col-lided you're the one, I have de-cided who's one of my kind], [Hey soul sister ain't that mister mister on the radio stereo, The way you move aint fair you know, Hey soul sister I don't want to miss a single thing you do], [To-night Hey-ay, Hey-ay-AY-ay-ay, Hey-ay-AY-ay-ay], [Just in ti-i-i-ime I'm so glad you have a one track mind like me, You gave my love dir-ection a game show love con-nection we can't deny-i-iii, I'm so obsessed my heart is bound to beat right outta my untrimmed chest, I believe in you like a virgin you're Ma-donna and I'm always gonna wanna, blow your mind], [Hey soul sister ain't that mister mister on the radio stereo, The way you move aint fair you know, Hey soul sister I don't want to miss a single ..."
